In [4]:
# load model 
import pickle
import torch as tc
from transformer_lens import HookedTransformer

from matplotlib import pyplot as plt
import numpy as np

def imshow(tensor, xaxis="", yaxis="", **kwargs):
    plt.imshow(tensor.detach().numpy(), cmap='RdBu', **kwargs)
    plt.xlabel(xaxis)
    plt.ylabel(yaxis)
    plt.show()

timestamp = '20240429143845'
with open(f'saved_models/{timestamp}.cfg', 'rb') as f:
    cfg = pickle.load(f)
# load model
model = HookedTransformer(cfg)
model.load_state_dict(tc.load(f'saved_models/{timestamp}.pt'))

<All keys matched successfully>

In [5]:
import itertools
vocab_size = cfg.d_vocab_out
input_size = cfg.d_vocab
def get_all_sequences(max_seq_len, vocab_size):
    # Generate all possible sequences of length n with vocabulary size m
    sequences = itertools.product(range(vocab_size), repeat=max_seq_len)
    sequences = [tc.tensor(seq) for seq in sequences]
    return sequences
sequences = get_all_sequences(input_size, vocab_size)
print(f'Generated {len(sequences)} sequences of length {input_size} with vocabulary size {vocab_size}')
sequences = tc.stack(sequences)
print(f'Stacked sequences shape: {sequences.shape}')

original_logits, cache = model.run_with_cache(sequences)

Generated 78125 sequences of length 7 with vocabulary size 5
Stacked sequences shape: torch.Size([78125, 7])


In [6]:
for param_name, param in cache.items():
    print(param_name, param.shape)

hook_embed torch.Size([78125, 7, 34])
hook_pos_embed torch.Size([78125, 7, 34])
blocks.0.hook_resid_pre torch.Size([78125, 7, 34])
blocks.0.attn.hook_q torch.Size([78125, 7, 1, 9])
blocks.0.attn.hook_k torch.Size([78125, 7, 1, 9])
blocks.0.attn.hook_v torch.Size([78125, 7, 1, 9])
blocks.0.attn.hook_attn_scores torch.Size([78125, 1, 7, 7])
blocks.0.attn.hook_pattern torch.Size([78125, 1, 7, 7])
blocks.0.attn.hook_z torch.Size([78125, 7, 1, 9])
blocks.0.hook_attn_out torch.Size([78125, 7, 34])
blocks.0.hook_resid_mid torch.Size([78125, 7, 34])
blocks.0.mlp.hook_pre torch.Size([78125, 7, 35])
blocks.0.mlp.hook_post torch.Size([78125, 7, 35])
blocks.0.hook_mlp_out torch.Size([78125, 7, 34])
blocks.0.hook_resid_post torch.Size([78125, 7, 34])
blocks.1.hook_resid_pre torch.Size([78125, 7, 34])
blocks.1.attn.hook_q torch.Size([78125, 7, 1, 9])
blocks.1.attn.hook_k torch.Size([78125, 7, 1, 9])
blocks.1.attn.hook_v torch.Size([78125, 7, 1, 9])
blocks.1.attn.hook_attn_scores torch.Size([78125, 1

In [31]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
from ipywidgets import interactive
from IPython.display import display

# Create a slider to control the value of i
slider = widgets.IntSlider(min=0, max=len(sequences)-1, value=0, description='i:')
#display(slider)

# Function to update the outputs based on the value of i
def update_outputs(i):
    print(sequences[i])
    #plt.gcf().clear()  # Clear the current figure
    # use cmap='RdBu' for better visualisation of negative values
    plt.set_cmap('RdBu')
    
    # plt.imshow(cache["hook_embed"][i])
    # plt.colorbar(shrink=0.3)
    # plt.title("hook_embed")
    # plt.show()
    
    # plt.imshow(cache["hook_pos_embed"][i])
    # plt.colorbar(shrink=0.3)
    # plt.title("hook_pos_embed")
    # plt.show()
    plt.imshow(cache["embedding: blocks.0.hook_resid_pre"][i])
    plt.colorbar(shrink=0.3)
    plt.title("blocks.0.hook_resid_pre")
    plt.show()
    
    plt.imshow(cache["blocks.0.hook_attn_out"][i])
    plt.colorbar(shrink=0.3)
    plt.title("blocks.0.hook_attn_out")
    plt.show()

    imshow(cache["pre", 0, "attn"][i])

    # imshow(cache["pre", 0, "mlp"][i])

    # imshow(cache["pre", 1, "attn"][i])

    # imshow(cache["pre", 1, "mlp"][i])
    # imshow(cache["pre", 2, "attn"][i])

    # imshow(cache["pre", 2, "mlp"][i])
    
    plt.tight_layout()
    plt.show()

# Connect the slider to the update_outputs function
# slider.observe(lambda change: update_outputs(change.new), names='value')


interactive_output = interactive(update_outputs, i=slider)
display(slider, interactive_output)

IntSlider(value=0, description='i:', max=78124)

interactive(children=(IntSlider(value=0, description='i:', max=78124), Output()), _dom_classes=('widget-intera…

In [ ]:
# TODO reverse engineer how the model works, start with the first layer and then check what each key does and how they relate to each other